In [1]:

from pathlib import Path
import torch
torch.cuda.empty_cache()
paths = [str(x) for x in Path('C:/langchain2/wiki/wiki_2/trext_data').glob('**/*.txt')]
paths

['C:\\langchain2\\wiki\\wiki_2\\trext_data\\Chikungunya.txt',
 'C:\\langchain2\\wiki\\wiki_2\\trext_data\\dengue.txt',
 'C:\\langchain2\\wiki\\wiki_2\\trext_data\\Fatigue.txt',
 'C:\\langchain2\\wiki\\wiki_2\\trext_data\\Fever.txt',
 'C:\\langchain2\\wiki\\wiki_2\\trext_data\\Headache.txt',
 'C:\\langchain2\\wiki\\wiki_2\\trext_data\\Malaria.txt',
 'C:\\langchain2\\wiki\\wiki_2\\trext_data\\Nausea.txt',
 'C:\\langchain2\\wiki\\wiki_2\\trext_data\\Rash.txt',
 'C:\\langchain2\\wiki\\wiki_2\\trext_data\\Tuberculosis.txt',
 'C:\\langchain2\\wiki\\wiki_2\\trext_data\\Typhoid_fever.txt',
 'C:\\langchain2\\wiki\\wiki_2\\trext_data\\Vomiting.txt',
 'C:\\langchain2\\wiki\\wiki_2\\trext_data\\who_dengue.txt']

In [2]:
# initialize an empty string to hold all the text
text = ""

# loop over the file paths and read each file into the text string
for path in paths:
    with open(path, 'r', encoding='utf-8') as f:
        text += f.read()


lower of one processed.txt

In [3]:
text = text.replace('\n', ' ')

In [4]:
text

' Chikungunya is an infection caused by the Chikungunya virus (CHIKV).[5][3] Symptoms include fever and joint pains.[2] These typically occur two to twelve days after exposure.[3] Other symptoms may include headache, muscle pain, joint swelling, and a rash.[2] Symptoms usually improve within a week; however, occasionally the joint pain may last for months or years.[2][6] The risk of death is around 1 in 1,000.[4] The very young, old, and those with other health problems are at risk of more severe disease.[2] The virus is spread between people by two types of mosquitos: Aedes albopictus and Aedes aegypti.[3] They mainly bite during the day.[7] The virus may circulate within a number of animals including birds and rodents.[3] Diagnosis is by either testing the blood for the virus\'s RNA or antibodies to the virus.[3] The symptoms can be mistaken for those of dengue fever and Zika fever.[3] It is believed most people become immune after a single infection.[2] The best means of prevention 

In [5]:
print("Length of text: {} characters".format(len(text)))

Length of text: 310883 characters


In [6]:
# split the text into smaller chunks
chunks = [text[i:i+1000] for i in range(0, len(text), 1000)]

In [7]:
num_chunks = len(chunks)
print("Number of chunks created:", num_chunks)

Number of chunks created: 311


In [8]:
from transformers import AutoTokenizer
import torch 

tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")

c:\Users\rifat\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [35]:
# create features for each chunk
features = []
for chunk in chunks:
    encoding = tokenizer.encode_plus(
        chunk,
        add_special_tokens=True,
        max_length=128,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_token_type_ids=True,
        return_tensors='pt'
    )
    features.append(encoding)

# stack the features
input_ids = torch.cat([f['input_ids'] for f in features], dim=0)
attention_mask = torch.cat([f['attention_mask'] for f in features], dim=0)
token_type_ids = torch.cat([f['token_type_ids'] for f in features], dim=0)


In [36]:
input_ids

tensor([[     0,   5585,  49314,  ...,   7304,   3181,      2],
        [     0, 128512,   7304,  ...,  92610,    297,      2],
        [     0,     36,  24243,  ...,  14361,     74,      2],
        ...,
        [     0,    104,    390,  ...,     70,  37896,      2],
        [     0,  16500,   3126,  ..., 179965,   8305,      2],
        [     0,      6,   2208,  ...,    168,   6261,      2]])

In [37]:
attention_mask

tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]])

In [38]:
encodings = {
    'input_ids': input_ids,
    'attention_mask': attention_mask,
    'labels': token_type_ids
}

In [39]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __len__(self):
        return self.encodings['input_ids'].shape[0]
    def __getitem__(self,i):
        return {key: tensor[i] for key, tensor in self.encodings.items()}

In [40]:
dataset = Dataset(encodings)

In [14]:
dataset

In [15]:
dataloader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True)

In [16]:
dataloader

In [17]:
# from transformers import RobertaConfig

In [18]:
# config = RobertaConfig(
#     vocab_size=30_000,
#     max_position_embeddings=514,
#     hidden_size=768,
#     num_attention_heads=12,
#     num_hidden_layers=4,
#     type_vocab_size=1


# )

In [19]:
from transformers import AutoModelForMaskedLM
# from transformers import RobertaForMaskedLM


In [20]:
# model=RobertaForMaskedLM(config=config)

In [21]:
# # device = torch.device('cpu')
# if torch.cuda.is_available():
#     device = torch.device('cuda')
#     print('Using gpu......................')
# else:
#     print('Using CPU')


In [22]:
device = torch.device('cpu')

In [23]:

model = AutoModelForMaskedLM.from_pretrained("C:/langchain2/wiki/wiki_2/trext_data/roberta_model_1")

In [24]:
# device = torch.device('cpu')


In [25]:
model.to(device)

XLMRobertaForMaskedLM(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
  

In [26]:
from transformers import AdamW
from tqdm.auto import tqdm


In [27]:
optim = AdamW(model.parameters(), lr=1e-5)

c:\Users\rifat\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [28]:
# from torch.cuda.amp import autocast, GradScaler

# batch_size = 1
# accumulation_steps = 12  # Accumulate gradients over 8 batches
# epoch = 4

# scaler = GradScaler()

# loop = tqdm(dataloader, leave=True)
# for i, batch in enumerate(loop):
#     if i % batch_size == 0:
#         optim.zero_grad()
    
#     input_ids = batch['input_ids'].to(device)
#     attention_mask = batch['attention_mask'].to(device)
#     labels = batch['labels'].to(device)
    
#     with autocast():
#         # Forward pass
#         outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
#         loss = outputs.loss
        
#         # Backward pass
#         scaler.scale(loss).backward()
        
#         if (i + 1) % accumulation_steps == 0:
#             scaler.step(optim)
#             scaler.update()
#             optim.zero_grad()
        
#         loop.set_description(f'Epoch:{epoch}')
#         loop.set_description(f'loss:{loss.item()}')

#         # Free up GPU memory after every batch
#         del input_ids, attention_mask, labels, outputs
#         torch.cuda.empty_cache()

# if (i + 1) % accumulation_steps != 0:
#     scaler.step(optim)
#     scaler.update()
#     optim.zero_grad()


In [29]:


batch_size = 1
epoch = 4
loop = tqdm(dataloader, leave=True)
for i, batch in enumerate(loop):
    if i % batch_size == 0:
        optim.zero_grad()
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    labels = batch['labels'].to(device)
    outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
    loss = outputs.loss
    loss.backward()
    if (i + 1) % batch_size == 0:
        optim.step()
        loop.set_description(f'Epoch:{epoch}')
        loop.set_description(f'loss:{loss.item()}')

    # Free up GPU memory after every batch
    del input_ids, attention_mask, labels, outputs
    torch.cuda.empty_cache()


loss:1.064368260195181e-09: 100%|██████████| 20/20 [08:43<00:00, 26.19s/it] 


In [30]:
model.save_pretrained('C:/langchain2/wiki/wiki_2/trext_data/roberta_model_1')

In [ ]:
# Preprocess your specific text using the same tokenizer
text = "On 5 December 2022 the European Medicines Agency approved Qdenga, a live tetravalent attenuated vaccine for adults, adolescents and kids from four years of age, produced by Takeda Pharmaceutical Company"
encoded_text = tokenizer(text, truncation=True, padding=True, return_tensors='pt')

# Classify the text
prediction = model.predict(encoded_text)[0][0]

# Interpret the prediction result
if prediction > 0.5:
    print("The text belongs to the positive class")
else:
    print("The text belongs to the negative class")


In [ ]:

# Define the input sentences
sentence_a = "Dengue is caused by "
sentence_b = "a bactria"

# Encode the input sentences
encoded_inputs = tokenizer(sentence_a, sentence_b, return_tensors='pt')

# Generate the prediction
model.eval()
with torch.no_grad():
    outputs = model(**encoded_inputs)

# Interpret the prediction
probs = torch.softmax(outputs[0], dim=1)
next_sentence_proba = probs.squeeze()[0].item()

# Print the predicted probability
print("Probability of next sentence:", next_sentence_proba)